# CHI@Edge: Using camera peripherals

Some of the CHI@Edge devices have attached camera modules (e.g., the Raspberry Pi V2 camera). You can reserve these devices and launch containers on them and specify an additional `device_profiles` argument to request that your container have access to a particular attached device/peripheral. The current list of supported peripherals is:

- `pi_camera`: for accessing the Raspberry Pi camera module from a container running on a Raspberry Pi.
- `jetson_csi_camera`: for accessing the Raspberry Pi camera module (or any other CSI-attached camera) from a container running on a Jetson.
- `pi_gpio`: for accessing the GPIO port of the Raspberry Pi.

In the following example we'll show how to use either the Raspberry Pi or the Jetson to capture camera data and stream it to the Jupyter notebook. We will create a container that exposes the camera stream data via an HTTP interface, but this is not the only way to access that data; the important part is that the container can read the device just as if the code was running directly on the host.

### Configuration

We'll set up some parameters that we'll use for both the lease request and the container create.

In [15]:
import chi
chi.use_site("CHI@Edge")
chi.set("project_name", "Chameleon")

# Choose your target platform for the demo
target_platform = "rpi"
# target_platform = "nano"

if target_platform == "rpi":
    # Currently it is not possible to make a reservation targeting devices having peripherals;
    # you must know the name of the device that has the peripheral physically connected.
    device_res_kwargs = {"device_name": "rpi3-01"}
    device_profiles = ["pi_camera"]
    container_kwargs = {}
elif target_platform == "nano":
    # Currently it is not possible to make a reservation targeting devices having peripherals;
    # you must know the name of the device that has the peripheral physically connected.
    device_res_kwargs = {"device_name": "nano-01"}
    device_profiles = ["jetson_csi_camera"]
    # Jetson Nano currently needs a custom Docker runtime for most things.
    container_kwargs = {"runtime": "nvidia"}
else:
    raise ValueError(f"Invalid target platform: {target_platform}")

Now using CHI@Edge:
URL: https://chi.edge.chameleoncloud.org
Location: University of Chicago, Chicago, Illinois, USA
Support contact: help@chameleoncloud.org


## Reserve a device with attached camera

In [2]:
from chi import lease
from chi import container

res = []
lease.add_device_reservation(res, count=1, **device_res_kwargs)
start, end = lease.lease_duration(days=2)
l = lease.create_lease(f"{target_platform}-camera-capture", reservations=res, start_date=start, end_date=end)

print("Lease created, waiting for it to start ...")

lease.wait_for_active(l["id"])

print("Lease active!")

Lease created, waiting for it to start ...
Lease active!


## Launch a container

For this container, we have a simple Flask HTTP server that will read camera data and serve it over a feed. There are two images, because each platform accesses the camera slightly differently:

- [diurnalist/rpi-video-streaming](https://hub.docker.com/repository/docker/diurnalist/rpi-video-streaming)
- [diurnalist/nano-video-streaming](https://hub.docker.com/repository/docker/diurnalist/nano-video-streaming)

These are both based on the [flask-video-streaming](https://github.com/miguelgrinberg/flask-video-streaming) example. [Here's the Dockerfile that creates the Raspberry Pi flavor](https://github.com/diurnalist/flask-video-streaming/blob/master/Dockerfile).

Notice we are exposing port 5000 via `exposed_ports`: this is the port the server will run on.

In [16]:
lease_id = lease.get_lease_id("rpi-camera-capture")

In [17]:
from chi import container, lease

print("Creating container ...")

camera_capture = container.create_container(
    f"{target_platform}-camera-capture", 
    image=f"diurnalist/{target_platform}-video-streaming:latest",
    device_profiles=["pi_camera"],
    exposed_ports=[5000],
    reservation_id=lease.get_device_reservation(l["id"]),
    **container_kwargs
)

print("Waiting for container to start ...")

# This won't return until our container is ready!
container.wait_for_active(camera_capture.uuid)

print("Done!")

Creating container ...
Waiting for container to start ...
Done!


## Associate public IP

In order to access the video feed over the internet from this Notebook, we need to attach a public IP address to our container.

In [18]:
public_ip = container.associate_floating_ip(camera_capture.uuid)
stream_url = f"http://{public_ip}:5000/video_feed"

## Access the video feed

Finally, we can use `ipywidgets` to directly display the current value of the stream! You could also download the image and convert it to an OpenCV image and perform further processing, or any number of other things.

In [19]:
import ipywidgets as widgets

stream = widgets.Image()
stream.format = "url"
stream.value = stream_url.encode("utf-8")
stream.width = 640
stream

Image(value=b'http://129.114.34.159:5000/video_feed', format='url', width='640')

In [20]:
container.destroy_container(camera_capture.uuid)